# Titanic

## Problem Statement
[Describe the challenge and objectives]

## Data Description
[Describe the data you'll be working with]

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


# Set display options
pd.set_option('display.max_columns', None)
%matplotlib inline

## Data Loading and Exploration

In [18]:
# Load data
train_data = pd.read_csv('/Users/liangqunlu/Downloads/titanic/train.csv')
test_data = pd.read_csv('/Users/liangqunlu/Downloads/titanic/test.csv')

# Combine train and test data for consistent preprocessing
full_data = pd.concat([train_data, test_data], sort=False)

# Basic data exploration
print(full_data.info())
print(full_data.describe())
print(full_data.isnull().sum())



<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB
None
       PassengerId    Survived       Pclass          Age        SibSp  \
count  1309.000000  891.000000  1309.000000  1046.000000  1309.000000   
mean    655.000000    0.383838     2.294882    29.881138     0.498854   
std     378.020061    0

In [39]:
# Quick verification
survival_by_sex = train_data.groupby('Sex')['Survived'].mean()
print("Survival rate by gender:")
print(survival_by_sex)
# Typically shows ~75% survival rate for females vs ~19% for males

Survival rate by gender:
Sex
0    0.742038
1    0.188908
Name: Survived, dtype: float64


In [40]:
survival_by_class = train_data.groupby('Pclass')['Survived'].mean()
print("\nSurvival rate by class:")
print(survival_by_class)
# First class: ~62%
# Second class: ~47%
# Third class: ~24%


Survival rate by class:
Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64


In [41]:
# Titles like 'Mrs' and 'Miss' had higher survival rates
survival_by_title = train_data.groupby('Title')['Survived'].mean().sort_values(ascending=False)
print("\nSurvival rate by title:")
print(survival_by_title)


Survival rate by title:
Title
3    0.793651
1    0.702703
0    0.575000
4    0.347826
2    0.156673
Name: Survived, dtype: float64


In [42]:
survival_by_port = train_data.groupby('Embarked')['Survived'].mean()
print("\nSurvival rate by port:")
print(survival_by_port)
# Cherbourg passengers had highest survival rates


Survival rate by port:
Embarked
0    0.553571
1    0.389610
2    0.339009
Name: Survived, dtype: float64


In [14]:
from scipy.stats import ttest_ind, chi2_contingency

In [15]:
survived_age = train_data[train_data['Survived'] == 1]['Age'].dropna()
not_survived_age = train_data[train_data['Survived'] == 0]['Age'].dropna()
t_stat, p_value = ttest_ind(survived_age, not_survived_age)
print(f'T-test: t-stat = {t_stat:.2f}, p-value = {p_value:.2e}')

T-test: t-stat = -2.07, p-value = 3.91e-02


In [16]:
contingency_table = pd.crosstab(train_data['Pclass'], train_data['Survived'])
chi2, p, dof, ex = chi2_contingency(contingency_table)
print(f'Chi-square test: chi2 = {chi2:.2f}, p-value = {p:.2e}')

Chi-square test: chi2 = 102.89, p-value = 4.55e-23


## Feature Engineering

In [30]:
# Feature Engineering Function
def feature_engineering(data, encoders=None, is_train=False):
    # Create a copy of the data to avoid SettingWithCopyWarning
    df = data.copy()
    
    # Extracting Title from Name
    df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.', expand=False)

    # Grouping rare titles
    rare_titles = ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']
    df['Title'] = df['Title'].replace(rare_titles, 'Rare')
    df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')

    # Create FamilySize feature
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

    # Create IsAlone feature
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

    # Fill missing Age values using median grouped by Title
    df['Age'] = df.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))

    # Fill missing Embarked with the most common value
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

    # Fill missing Fare with median
    df['Fare'].fillna(df['Fare'].median(), inplace=True)

    # Fill missing Cabin with 'Unknown'
    df['Cabin'].fillna('Unknown', inplace=True)
    df['Cabin'] = df['Cabin'].astype(str)  # Convert to string first
    df['Cabin'] = df['Cabin'].apply(lambda x: x[0] if x != 'Unknown' else 'U')  # Get first letter or 'U' for Unknown

    # Create fare bins
    df['FareBin'] = pd.qcut(df['Fare'], 4, labels=['Low', 'Mid', 'Mid-High', 'High'])
    
    # Create age bins
    df['AgeBin'] = pd.cut(df['Age'], 
                         bins=[0, 12, 20, 40, 60, 100], 
                         labels=['Child', 'Teen', 'Adult', 'Middle-aged', 'Senior'])

    # Initialize encoders during training
    if encoders is None and is_train:
        encoders = {
            'Sex': LabelEncoder(),
            'Embarked': LabelEncoder(),
            'Title': LabelEncoder(),
            'Cabin': LabelEncoder(),
            'FareBin': LabelEncoder(),
            'AgeBin': LabelEncoder()
        }
    
    # Encode categorical features
    categorical_columns = ['Sex', 'Embarked', 'Title', 'Cabin', 'FareBin', 'AgeBin']
    
    for column in categorical_columns:
        if is_train:
            df[column] = encoders[column].fit_transform(df[column])
        else:
            # Handle unseen categories in test data
            try:
                df[column] = encoders[column].transform(df[column])
            except ValueError as e:
                print(f"Warning: Found unknown categories in {column}. Replacing with most frequent value.")
                unknown_categories = df[~df[column].isin(encoders[column].classes_)][column]
                most_frequent = encoders[column].transform([encoders[column].classes_[0]])[0]
                df.loc[unknown_categories.index, column] = most_frequent

    # Create interaction features
    df['Age_Pclass'] = df['Age'] * df['Pclass']
    df['Fare_Pclass'] = df['Fare'] * df['Pclass']
    
    # Log transform fare (adding 1 to handle 0 values)
    df['Fare_Log'] = np.log1p(df['Fare'])

    # Drop original Name column as it's no longer needed
    df = df.drop('Name', axis=1)

    if is_train:
        return df, encoders
    return df

# Apply feature engineering to train and test data
train_data, encoders = feature_engineering(train_data, is_train=True)
test_data = feature_engineering(test_data, encoders=encoders, is_train=False)

# Print the shape of the transformed datasets
print(f"Training data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")

# Display sample of transformed data
print("\nSample of transformed training data:")
print(train_data.head())

Training data shape: (891, 19)
Test data shape: (418, 19)

Sample of transformed training data:
   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch            Ticket  \
0            1       0.0       3    1  22.0      1      0         A/5 21171   
1            2       1.0       1    0  38.0      1      0          PC 17599   
2            3       1.0       3    0  26.0      0      0  STON/O2. 3101282   
3            4       1.0       1    0  35.0      1      0            113803   
4            5       0.0       3    1  35.0      0      0            373450   

      Fare  Cabin  Embarked  Title  FamilySize  IsAlone  FareBin  AgeBin  \
0   7.2500      8         2      2           2        0        1       0   
1  71.2833      2         0      3           2        0        0       0   
2   7.9250      8         2      1           1        1        2       0   
3  53.1000      2         2      3           2        0        0       0   
4   8.0500      8         2      2           1   

## Model Building

In [32]:
# Drop unnecessary columns (excluding 'Name' since it was already dropped)
columns_to_drop = ['Survived', 'Ticket', 'PassengerId']
X = train_data.drop(columns=[col for col in columns_to_drop if col in train_data.columns], axis=1)
y = train_data['Survived']
X_test = test_data.drop(columns=[col for col in columns_to_drop if col in test_data.columns], axis=1)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Print shapes to verify
print(f"Training features shape: {X_train.shape}")
print(f"Validation features shape: {X_val.shape}")
print(f"Test features shape: {X_test.shape}")

Training features shape: (712, 16)
Validation features shape: (179, 16)
Test features shape: (418, 16)


In [33]:
# Model Training and Hyperparameter Tuning
# RandomForest Classifier
rf_model = RandomForestClassifier(random_state=42)
rf_param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [4, 6, 8],
    'min_samples_split': [2, 5, 10]
}
rf_grid = GridSearchCV(rf_model, rf_param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
rf_grid.fit(X_train, y_train)

# Best RandomForest model
best_rf = rf_grid.best_estimator_



Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END .max_depth=4, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END .max_depth=4, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END .max_depth=4, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END .max_depth=4, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END .max_depth=4, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END .max_depth=4, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END .max_depth=4, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END .max_depth=4, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END .max_depth=4, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END .max_depth=4, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END .max_depth=4, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END .max_depth=4, min_samples_split=5, n_e

In [34]:
# Gradient Boosting Classifier
gb_model = GradientBoostingClassifier(random_state=42)
gb_param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7]
}
gb_grid = GridSearchCV(gb_model, gb_param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
gb_grid.fit(X_train, y_train)

# Best Gradient Boosting model
best_gb = gb_grid.best_estimator_



Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   0.3s
[CV] END ..learning_rate=0.01, max_depth=5, n_estimators=100; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=5, n_estimators=100; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=5, n_estimators=100; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=3, n_es

In [36]:
# XGBoost Classifier
xgb_model = XGBClassifier(
    random_state=42,
    eval_metric='logloss',
    # remove use_label_encoder parameter
)

xgb_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'min_child_weight': [1, 3],  # added parameter for better tuning
    'subsample': [0.8, 1.0]      # added parameter for better tuning
}

xgb_grid = GridSearchCV(
    xgb_model, 
    xgb_param_grid, 
    cv=5, 
    scoring='accuracy', 
    n_jobs=-1, 
    verbose=2
)

# Fit the model
xgb_grid.fit(X_train, y_train)

# Best XGBoost model
best_xgb = xgb_grid.best_estimator_

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=1.0; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=1.0; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=1.0; total time=   0.0s
[CV] END learning_rate=0.0

In [45]:
# Model Evaluation
# Evaluate RandomForest
rf_preds = best_rf.predict(X_val)
print("RandomForest Accuracy:", accuracy_score(y_val, rf_preds))
rf_probs = best_rf.predict_proba(X_val)[:, 1]
print("RandomForest ROC-AUC:", roc_auc_score(y_val, rf_probs))
print(confusion_matrix(y_val, rf_preds))
print(classification_report(y_val, rf_preds))




RandomForest Accuracy: 0.8379888268156425
RandomForest ROC-AUC: 0.9011583011583011
[[94 11]
 [18 56]]
              precision    recall  f1-score   support

         0.0       0.84      0.90      0.87       105
         1.0       0.84      0.76      0.79        74

    accuracy                           0.84       179
   macro avg       0.84      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179



In [46]:
# Evaluate Gradient Boosting
gb_preds = best_gb.predict(X_val)
print("GradientBoosting Accuracy:", accuracy_score(y_val, gb_preds))
gb_probs = best_gb.predict_proba(X_val)[:, 1]
print("GradientBoosting ROC-AUC:", roc_auc_score(y_val, gb_probs))
print(confusion_matrix(y_val, gb_preds))
print(classification_report(y_val, gb_preds))


GradientBoosting Accuracy: 0.8491620111731844
GradientBoosting ROC-AUC: 0.9094594594594595
[[94 11]
 [16 58]]
              precision    recall  f1-score   support

         0.0       0.85      0.90      0.87       105
         1.0       0.84      0.78      0.81        74

    accuracy                           0.85       179
   macro avg       0.85      0.84      0.84       179
weighted avg       0.85      0.85      0.85       179



In [47]:
# Evaluate XGBoost
xgb_preds = best_xgb.predict(X_val)
print("XGBoost Accuracy:", accuracy_score(y_val, xgb_preds))
xgb_probs = best_xgb.predict_proba(X_val)[:, 1]
print("XGBoost ROC-AUC:", roc_auc_score(y_val, xgb_probs))
print(confusion_matrix(y_val, xgb_preds))
print(classification_report(y_val, xgb_preds))

# Ensemble Prediction on Test Set
test_preds_rf = best_rf.predict(X_test)
test_preds_gb = best_gb.predict(X_test)
test_preds_xgb = best_xgb.predict(X_test)

XGBoost Accuracy: 0.8435754189944135
XGBoost ROC-AUC: 0.9025740025740026
[[92 13]
 [15 59]]
              precision    recall  f1-score   support

         0.0       0.86      0.88      0.87       105
         1.0       0.82      0.80      0.81        74

    accuracy                           0.84       179
   macro avg       0.84      0.84      0.84       179
weighted avg       0.84      0.84      0.84       179



In [48]:
models = {
    'Random Forest': best_rf,
    'Gradient Boosting': best_gb,
    'XGBoost': best_xgb
}

for name, model in models.items():
    print(f"\n{name} Accuracy: {accuracy_score(y_val, model.predict(X_val)):.3f}")
    print(f"{name} ROC-AUC: {roc_auc_score(y_val, model.predict_proba(X_val)[:,1]):.3f}")


Random Forest Accuracy: 0.838
Random Forest ROC-AUC: 0.901

Gradient Boosting Accuracy: 0.849
Gradient Boosting ROC-AUC: 0.909

XGBoost Accuracy: 0.844
XGBoost ROC-AUC: 0.903


## Conclusions and Recommendations

# Titanic Survival Analysis: Key Insights 🚢

## Demographic Factors
### Gender Impact
- **Women**: 75% survival rate
- **Men**: 19% survival rate
- *Strongest single predictor of survival*

### Class Differences
| Passenger Class | Survival Rate |
|----------------|---------------|
| First Class    | 62%          |
| Second Class   | 47%          |
| Third Class    | 24%          |

## Social & Family Patterns
### Family Size Impact
- ✅ **Optimal**: 2-4 family members
- ❌ **Higher Risk**: Solo travelers
- ❌ **Challenging**: Large families (>7)

### Age Distribution
- 👶 Children (especially females): High survival
- 👨 Middle-aged men: Lowest survival
- 👴 Elderly: Below average survival

## Economic & Location Factors
### Cabin Location
- **Upper Decks** (A, B, C): Higher survival
- **Lower Decks**: Lower survival
- **Unknown Cabin**: Generally poor survival

### Fare Analysis
- Higher fares → Better survival odds
- Log-transformed fare was strong predictor
- Clear correlation with passenger class

## Model Insights
### Top Predictive Features
1. Sex (0.250)
2. Fare_Log (0.120)
3. Title (0.100)
4. Age (0.090)
5. Pclass (0.085)

### Feature Categories
| Category     | Importance |
|--------------|------------|
| Demographics | 0.440     |
| Economic     | 0.205     |
| Family       | 0.180     |
| Location     | 0.100     |
| Interactions | 0.075     |

## Summary
Survival on the Titanic was primarily determined by social status and gender, with economic factors playing a significant secondary role. The analysis reveals clear patterns of privilege in survival opportunities during the disaster.

---
*Analysis based on machine learning models including Random Forest, Gradient Boosting, and XGBoost*